# Tarea 5
- Ivan Morales, 10 de Octubre de 2025

Realizaremos dos experimentos, cada uno con child experiments

    - Gradient Boost
    - Random Forest

### Importar las librerias necesarias y establecer conexion con Databricks

In [2]:
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

load_dotenv(override=True) # Cargar las variables de entorno desde el archivo .env
EXPERIMENT_NAME = "/Users/ivan.morales@iteso.mx/nyc-taxi-experiments" 

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

Preparacion de los datos

In [3]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

df_train_ene = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_train_feb = read_dataframe('../data/green_tripdata_2025-02.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-03.parquet')

In [4]:
df_train_ene.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44218 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               44218 non-null  int32         
 1   lpep_pickup_datetime   44218 non-null  datetime64[us]
 2   lpep_dropoff_datetime  44218 non-null  datetime64[us]
 3   store_and_fwd_flag     42295 non-null  object        
 4   RatecodeID             42295 non-null  float64       
 5   PULocationID           44218 non-null  object        
 6   DOLocationID           44218 non-null  object        
 7   passenger_count        42295 non-null  float64       
 8   trip_distance          44218 non-null  float64       
 9   fare_amount            44218 non-null  float64       
 10  extra                  44218 non-null  float64       
 11  mta_tax                44218 non-null  float64       
 12  tip_amount             44218 non-null  float64       
 13  tolls_

In [5]:
df_train = pd.concat([df_train_ene, df_train_feb])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88436 entries, 0 to 46620
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               88436 non-null  int32         
 1   lpep_pickup_datetime   88436 non-null  datetime64[us]
 2   lpep_dropoff_datetime  88436 non-null  datetime64[us]
 3   store_and_fwd_flag     84590 non-null  object        
 4   RatecodeID             84590 non-null  float64       
 5   PULocationID           88436 non-null  object        
 6   DOLocationID           88436 non-null  object        
 7   passenger_count        84590 non-null  float64       
 8   trip_distance          88436 non-null  float64       
 9   fare_amount            88436 non-null  float64       
 10  extra                  88436 non-null  float64       
 11  mta_tax                88436 non-null  float64       
 12  tip_amount             88436 non-null  float64       
 13  tolls_

Concatemos los datasets de enero y febrero para construir una base mas robusta para validar con los datos de marzo.

Feature Engineering + One Hot Encoding, pipeline simple inicial

In [6]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los dataset como objetos de *mlflow* para poderlos trackear

In [7]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01&02")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-03")

# Modelos

In [8]:
# XGBOOST 
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [12]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

Flujo de Busqueda

In [ ]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

In [14]:
model_name = "workspace.default.nyc-taxi-model-xgboost-tarea-5"

In [ ]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name=model_name
)

Asignamos alias

In [16]:
from mlflow import MlflowClient

client = MlflowClient()

In [17]:
model_version = result.version
new_alias = "Challenger"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

In [ ]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

Obtenemos el modelo @Challenger

In [ ]:
import mlflow.pyfunc

model_version_uri = f"models:/{model_name}@Challenger"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

# Gradient Boost

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
from mlflow.models import infer_signature
import mlflow
import optuna
import math

In [17]:
def objective(trial: optuna.trial.Trial):
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "l2_regularization": trial.suggest_float("l2_regularization", math.exp(-6), math.exp(-1), log=True),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 5, 50),
    }

    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "sklearn-hist-gradient-boosting")
        mlflow.log_params(params)

        model = HistGradientBoostingRegressor(
            **params,
            max_iter=100,                
            early_stopping=True,         
            validation_fraction=0.1,     
            n_iter_no_change=10,         
            random_state=42,
            loss='squared_error'
        )

        # --- CAMBIO CLAVE: Convertir a array denso ---
        X_train_dense = X_train.toarray()
        X_val_dense = X_val.toarray()
        
        model.fit(X_train_dense, y_train)

        # --- CAMBIO CLAVE: Predecir sobre el array denso ---
        y_pred = model.predict(X_val_dense)
        rmse = root_mean_squared_error(y_val, y_pred) 

        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("best_n_estimators", model.n_iter_)

        # --- CAMBIO CLAVE: Usar el array denso para la firma y el ejemplo ---
        signature = infer_signature(X_val_dense, y_pred)
        
        mlflow.sklearn.log_model(
            model,
            "model",
            input_example=X_val_dense[:5], # Usar el denso
            signature=signature
        )

    return rmse

In [18]:
mlflow.sklearn.autolog(log_models=False)
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

with mlflow.start_run(run_name="HistGradientBoosting Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10)

    best_params = study.best_params
    best_params["max_depth"] = int(best_params["max_depth"])
    mlflow.log_params(best_params)

    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "sklearn-hist-gradient-boosting",
        "feature_set_version": 1,
    })

    # --- ENTRENAMIENTO FINAL ---
    final_model = HistGradientBoostingRegressor(
        **best_params,
        max_iter=100,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=42,
        loss='squared_error'
    )
    
    # --- CAMBIO CLAVE: Convertir a array denso ---
    X_train_dense = X_train.toarray()
    X_val_dense = X_val.toarray()
    
    final_model.fit(X_train_dense, y_train)

    # --- CAMBIO CLAVE: Predecir sobre el array denso ---
    y_pred = final_model.predict(X_val_dense)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("best_n_estimators", final_model.n_iter_)

    # --- Guardado de artefactos (Tu lógica de 'dv' está bien) ---
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # --- Signature y loggeo (Tu lógica original ya usaba .toarray() aquí, perfecto) ---
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val_dense[:5], columns=feature_names) # Usar denso
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(
        final_model,
        "model",
        input_example=input_example,
        signature=signature
    )

[I 2025-11-04 13:33:11,181] A new study created in memory with name: no-name-061cba7f-8c43-468d-932f-87ccec797280
2025/11/04 13:33:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:33:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/11/04 13:33:42 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:33:46,002] Trial 0 finished with value: 6.454844990021286 and parameters: {'max_depth': 40, 'learning_rate': 0.8625543817410922, 'l2_regularization': 0.09632472041968834, 'min_samples_leaf': 32}. Best is trial 0 with value: 6.454844990021286.


🏃 View run zealous-squirrel-13 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/ebac4e83ef204e79902cf79adeb9a7cc
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


2025/11/04 13:33:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:34:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/11/04 13:34:05 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:34:07,241] Trial 1 finished with value: 6.3027164154240545 and parameters: {'max_depth': 19, 'learning_rate': 0.07949855457041166, 'l2_regularization': 0.0033140577330962573, 'min_samples_leaf': 44}. Best is trial 1 with value: 6.3027164154240545.


...

2025/11/04 13:36:43 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:36:46,087] Trial 9 finished with value: 6.2945156386472645 and parameters: {'max_depth': 33, 'learning_rate': 0.06673780747204877, 'l2_regularization': 0.0758623422350637, 'min_samples_leaf': 25}. Best is trial 3 with value: 6.292915183828678.


🏃 View run languid-hare-361 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/9cf55fbf3b7d4f70ae8f55539ac4d0b9
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


2025/11/04 13:36:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:37:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but HistGradientBoostingRegressor was fitted without feature names
  warnings.warn(
2025/11/04 13:37:05 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run HistGradientBoosting Hyperparameter Optimization (Optuna) at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/1b4d7f629bb84f70add559d1d98143a8
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


In [19]:
model_name = "workspace.default.nyc-taxi-model-gradient-boosting-tarea-5"

In [20]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name=model_name
)

Successfully registered model 'workspace.default.nyc-taxi-model-gradient-boosting-tarea-5'.
2025/11/04 13:40:41 WARNING mlflow.tracking._model_registry.fluent: Run with id 1b4d7f629bb84f70add559d1d98143a8 has no artifacts at artifact path 'model', registering model based on models:/m-7a155231fbef4b6db2ee812925422748 instead


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model-gradient-boosting-tarea-5'.


# RandomForest

In [21]:
from sklearn.ensemble import RandomForestRegressor

def objective(trial: optuna.trial.Trial):
    # --- CAMBIO: Hiperparámetros para RandomForestRegressor ---
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 300),
        "max_depth": trial.suggest_int("max_depth", 5, 50),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_float("max_features", 0.1, 1.0),
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        # --- CAMBIO ---
        mlflow.set_tag("model_family", "sklearn-rf")
        mlflow.log_params(params)

        # --- CAMBIO: Instanciar el modelo RandomForestRegressor ---
        model = RandomForestRegressor(
            **params,
            random_state=42,
            n_jobs=-1  # ¡Usar todos los cores! RF paraleliza muy bien
        )

        # --- CAMBIO: Entrenamiento directo sobre datos sparse ---
        # (RandomForest SÍ acepta X_train sparse, no se necesita .toarray())
        model.fit(X_train, y_train)

        # --- CAMBIO: Predicción sobre datos sparse ---
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)
        # (No hay 'best_n_estimators' ya que RF no usa early stopping)

        # --- CAMBIO: Signature y loggeo ---
        # (X_val es sparse, lo cual es aceptado)
        signature = infer_signature(X_val, y_pred)

        mlflow.sklearn.log_model(
            model,
            "model",
            input_example=X_val[:5], # Pasamos un slice del sparse
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

In [22]:
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna (Sin cambios)
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización
# ------------------------------------------------------------
# --- CAMBIO: Nombre del Run ---
with mlflow.start_run(run_name="RandomForest Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=10) # Llama a la nueva 'objective'

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # --- CAMBIO: Asegurar que los 'int' sean 'int' ---
    best_params["n_estimators"] = int(best_params["n_estimators"])
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["min_samples_split"] = int(best_params["min_samples_split"])
    best_params["min_samples_leaf"] = int(best_params["min_samples_leaf"])
    
    mlflow.log_params(best_params)

    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "sklearn-rf", # --- CAMBIO ---
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # Entrenar un modelo FINAL con los mejores hiperparámetros
    # --------------------------------------------------------
    # --- CAMBIO: Instanciar y entrenar con RandomForestRegressor ---
    final_model = RandomForestRegressor(
        **best_params,
        random_state=42,
        n_jobs=-1
    )
    
    # --- CAMBIO: .fit() directo sobre datos sparse ---
    final_model.fit(X_train, y_train)

    # Evaluar y registrar la métrica final en validación (X_val)
    y_pred = final_model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    # (No hay 'best_n_estimators' para loggear)

    # --------------------------------------------------------
    # Guardar artefactos adicionales (Sin cambios)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # --------------------------------------------------------
    # Signature y loggeo del modelo final
    # --------------------------------------------------------
    
    # --- (Tu lógica original es la mejor práctica aquí) ---
    # Para la 'signature' y el 'input_example' del modelo FINAL,
    # sigue siendo buena idea usar un DataFrame denso de Pandas
    # para que sea legible en la UI de MLflow.
    
    feature_names = dv.get_feature_names_out()
    # Usamos .toarray() SOLO para este ejemplo de 5 filas
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names) 
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(
        final_model,
        "model",
        input_example=input_example,
        signature=signature
    )

[I 2025-11-04 13:45:07,447] A new study created in memory with name: no-name-5a0aa452-60c8-4efa-b938-0fa094625c29
2025/11/04 13:45:20 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:45:26 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/11/04 13:45:26 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:45:58,136] Trial 0 finished with value: 6.399926016644729 and parameters: {'n_estimators': 118, 'max_depth': 48, 'min_samples_split': 15, 'min_samples_leaf': 12, 'max_features': 0.24041677639819287}. Best is trial 0 with value: 6.399926016644729.


🏃 View run abundant-roo-57 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/e79177f83b404431856585dc15042e9f
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


2025/11/04 13:46:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:46:07 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


2025/11/04 13:46:08 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:46:10,869] Trial 1 finished with value: 6.28765796156506 and parameters: {'n_estimators': 55, 'max_depth': 7, 'min_samples_split': 18, 'min_samples_leaf': 13, 'max_features': 0.737265320016441}. Best is trial 1 with value: 6.28765796156506.


...

2025/11/04 13:50:09 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-11-04 13:50:33,045] Trial 9 finished with value: 6.306265095920583 and parameters: {'n_estimators': 202, 'max_depth': 19, 'min_samples_split': 11, 'min_samples_leaf': 11, 'max_features': 0.26636900997297436}. Best is trial 1 with value: 6.28765796156506.


🏃 View run victorious-pig-524 at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/6b55c748f70d4fc3aa3b86c1b4e458cf
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


2025/11/04 13:50:37 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/04 13:50:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
2025/11/04 13:50:43 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run RandomForest Hyperparameter Optimization (Optuna) at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596/runs/5e60bb87020a4cab962a4838e5be7b38
🧪 View experiment at: https://dbc-ed7122c4-c5b7.cloud.databricks.com/ml/experiments/2414141065980596


In [23]:
model_name = "workspace.default.nyc-taxi-model-random-forest-tarea-5"

In [24]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name=model_name
)

Successfully registered model 'workspace.default.nyc-taxi-model-random-forest-tarea-5'.
2025/11/04 13:52:56 WARNING mlflow.tracking._model_registry.fluent: Run with id 5e60bb87020a4cab962a4838e5be7b38 has no artifacts at artifact path 'model', registering model based on models:/m-ed68dd1fc4134f5db4bcb7fd980b0588 instead


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model-random-forest-tarea-5'.


# Campeonato

In [27]:
import mlflow
import pandas as pd
from sklearn.metrics import root_mean_squared_error
from mlflow.tracking import MlflowClient
# Asumo que 'dv' (el DictVectorizer entrenado) y 'read_dataframe' 
# ya están definidos en celdas anteriores.

# --- 1. Configuración ---
mlflow.set_registry_uri("databricks-uc")
model_name = "workspace.default.nyc-taxi-model"
aliases_to_test = ["champion", "challenger_gb", "challenger_rf"]

# --- 2. Preparar Datos de Prueba (Marzo 2025) ---
print("Cargando y procesando datos de prueba (Marzo 2025)...")

# Cargamos los datos de marzo como nuestro set de prueba
df_test = read_dataframe('../data/green_tripdata_2025-03.parquet')
y_test = df_test['duration'].values

# --- ¡ESTA ES LA PARTE IMPORTANTE! ---
# Debemos usar la MISMA lógica de preprocesamiento que en el ENTRENAMIENTO

# ASUNCIÓN: 'dv' es el DictVectorizer ya entrenado de tu celda anterior
if 'dv' not in locals():
    print("ERROR: 'dv' no está definido. Asegúrate de correr la celda que entrena el DictVectorizer.")

# 1. Definir las features (igual que en X_train)
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# 2. Convertir a diccionarios (igual que en X_train)
test_dicts = df_test[categorical + numerical].to_dict(orient='records')

# 3. Transformar con el 'dv' ya entrenado
X_test_sparse = dv.transform(test_dicts)

# 4. Obtener nombres de features del 'dv'
feature_names = dv.get_feature_names_out()

# 5. Convertir a DataFrame denso para el método .predict() estandarizado
X_test_df = pd.DataFrame(X_test_sparse.toarray(), columns=feature_names)

print(f"Datos de prueba listos. Shape: {X_test_df.shape}")


# --- 3. Bucle de Evaluación ---
results = {}
model_versions = {}

print(f"\nIniciando evaluación del modelo: {model_name}")

for alias in aliases_to_test:
    print(f"--- Evaluando: {alias} ---")
    try:
        model_uri = f"models:/{model_name}@{alias}"
        model = mlflow.pyfunc.load_model(model_uri)
        
        client = MlflowClient()
        version_details = client.get_model_version_by_alias(model_name, alias)
        model_versions[alias] = version_details.version
        print(f"  Cargando Versión: {version_details.version}")

        # Realizar predicciones sobre el DataFrame de prueba
        y_pred = model.predict(X_test_df)
        
        rmse = root_mean_squared_error(y_test, y_pred)
        results[alias] = rmse
        print(f"  RMSE de {alias} (Versión {version_details.version}): {rmse:.4f}")

    except Exception as e:
        print(f"  No se pudo cargar o evaluar {alias}: {e}")

# --- 4. Comparar Resultados ---
print("\n--- 🏆 Resultados del Campeonato ---")
comparison_df = pd.DataFrame.from_dict(results, orient="index", columns=["RMSE"])
comparison_df["Version"] = comparison_df.index.map(model_versions)
comparison_df = comparison_df.sort_values("RMSE")

print(comparison_df)

winner_alias = comparison_df.index[0]
winner_version = comparison_df.iloc[0]["Version"]
winner_rmse = comparison_df.iloc[0]["RMSE"]

print(f"\nEl ganador es '{winner_alias}' (Versión {winner_version}) con un RMSE de: {winner_rmse:.4f}")

# --- 5. Promover al Ganador (Ejecutar si estás seguro) ---
# client = MlflowClient()
# new_champion_alias = winner_alias 

# if new_champion_alias != "Champion":
#     print(f"\nPromoviendo '{new_champion_alias}' (Versión {winner_version}) a 'Champion'...")
    
#     client.set_registered_model_alias(
#         name=model_name,
#         alias="Champion",
#         version=winner_version
#     )
#     print("¡Promoción completada!")
    
#     try:
#         client.delete_registered_model_alias(
#             name=model_name,
#             alias=new_champion_alias
#         )
#         print(f"Alias '{new_champion_alias}' limpiado.")
#     except Exception as e:
#         pass
# else:
#     print(f"\nEl 'Champion' (Versión {winner_version}) sigue siendo el mejor modelo.")

Cargando y procesando datos de prueba (Marzo 2025)...
Datos de prueba listos. Shape: (48336, 452)

Iniciando evaluación del modelo: workspace.default.nyc-taxi-model
--- Evaluando: champion ---


c:\Users\ivanm\5to Semestre\Proyecto de Ciencia de Datos\nyc-taxi-predictions-2025\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [14:27:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


  Cargando Versión: 2
  RMSE de champion (Versión 2): 66.4154
--- Evaluando: challenger_gb ---


  Cargando Versión: 4
  RMSE de challenger_gb (Versión 4): 5.7557
--- Evaluando: challenger_rf ---


  Cargando Versión: 3
  RMSE de challenger_rf (Versión 3): 6.0298

--- 🏆 Resultados del Campeonato ---
                    RMSE Version
challenger_gb   5.755702       4
challenger_rf   6.029828       3
champion       66.415430       2

El ganador es 'challenger_gb' (Versión 4) con un RMSE de: 5.7557


Vemos que con creces el mejor modelo es el que realizamos con Gradient Boost, mejora considerablemente respecto a Random Forest y ni se diga del modelo Champion anterior, aunque debemos de considerar que este ultimo solo estaba entrenado con los datosd enero, seria una buena idea probar a entrenar el modelo de nuevo con dichos datos para que compitan en un terreno justo.